In [1]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder,MultiLabelBinarizer
from sklearn.feature_selection import SelectKBest, chi2

# Data Processing In Main Notebook

In [2]:
df = pd.read_csv('secondary_data.csv',sep=';', on_bad_lines='skip')
df = df.rename(columns={"Cap-surface": "cap-surface"})

df = df.drop(columns=['spore-print-color',
                     'has-ring'])

#Binary Encoding
binary_mapping = {'[t]': 1, '[f]': 0}
df['does-bruise-or-bleed'] = df['does-bruise-or-bleed'].map(binary_mapping)
binary_mapping = {'p': 1, 'e': 0}
df['class'] = df['class'].map(binary_mapping)

df = df.drop(columns=['stem-root', 'stem-surface', 'veil-type', 'veil-color'])

df['gill-attachment'].fillna('[?]', inplace=True)
df['ring-type'].fillna('[?]', inplace=True)
df['cap-surface'].fillna('[?]', inplace=True)
df['gill-spacing'].fillna('[?]', inplace=True)

columns_to_apply_multilabeling = ['cap-shape','cap-surface','cap-color','gill-attachment', 'gill-spacing', 'gill-color','stem-color','ring-type','season']

for col in columns_to_apply_multilabeling:
    mlb = MultiLabelBinarizer()
    column = df[col].apply(lambda x: x.strip('[]').split(', '))
    
    encoded_data = mlb.fit_transform(column)
    encoded_df = pd.DataFrame(encoded_data, columns=mlb.classes_+"_"+col)
    df = df.drop(col, axis=1)
    df = pd.concat([df, encoded_df], axis=1)

X = df.drop(columns = {'habitat','cap-diameter','stem-height','stem-width'})

##drop all columns with NAN
X = X.dropna(axis=1)

y = df['habitat']

selector = SelectKBest(score_func=chi2, k=10)  # Specify the desired number of features
X_selected = selector.fit_transform(X, y)
selected_feature_indices = selector.get_support(indices=True)
selected_feature_names = X.columns[selected_feature_indices]
X_selected_df = pd.DataFrame(X_selected, columns=selected_feature_names)

#Encoding y
mlb = MultiLabelBinarizer()

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

C:\Users\Neil\AppData\Local\Temp\ipykernel_18204\3687707237.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['gill-attachment'].fillna('[?]', inplace=True)
C:\Users\Neil\AppData\Local\Temp\ipykernel_18204\3687707237.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

# Random Forest

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300], 
    'max_depth': [None, 10, 20, 30],  
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4]     
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best cross-validation score achieved: {grid_search.best_score_}")

# Use the best estimator to make predictions on the test set
best_estimator = grid_search.best_estimator_
y_pred = best_estimator.predict(X_test)

from sklearn.metrics import accuracy_score
print(f"Test set accuracy: {accuracy_score(y_test, y_pred)}")

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters found: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Best cross-validation score achieved: 0.8340804421246546
Test set accuracy: 0.828393646635009


In [5]:
from joblib import dump, load

model_filename = 'random_forest_model.model'
dump(best_estimator, model_filename)

['random_forest_model.model']

In [6]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


y_train_pred = best_estimator.predict(X_train)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91      8856
           1       0.72      0.72      0.72      1583
           2       0.44      0.45      0.45       409
           3       0.54      0.50      0.52       639
           4       0.54      0.53      0.53       566
           5       0.53      0.62      0.57        76
           6       0.35      0.29      0.32        24
           7       1.00      1.00      1.00        61

    accuracy                           0.83     12214
   macro avg       0.63      0.63      0.63     12214
weighted avg       0.83      0.83      0.83     12214

              precision    recall  f1-score   support

           0       0.92      0.93      0.93     35353
           1       0.79      0.79      0.79      6360
           2       0.54      0.57      0.55      1592
           3       0.65      0.62      0.63      2529
           4       0.70      0.66      0.68      2354
           5       0.58 